In [1]:

import os, sys, json, math, shutil, random, subprocess
from pathlib import Path
from datetime import datetime

import yaml
import numpy as np
import cv2
import torch
from tqdm import tqdm

try:
    from ultralytics import YOLO
    import ultralytics
    ULTRA_VER = ultralytics.__version__
except Exception:
    ULTRA_VER = None

def set_seed(seed=42):
    random.seed(seed); np.random.seed(seed)
    torch.manual_seed(seed); torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)
print("Torch:", torch.__version__, "| CUDA:", torch.version.cuda, "| Ultralytics:", ULTRA_VER)
print("GPU available:", torch.cuda.is_available(),
      ("| GPU: " + torch.cuda.get_device_name(0)) if torch.cuda.is_available() else "")


Torch: 2.5.1+cu121 | CUDA: 12.1 | Ultralytics: 8.3.0
GPU available: True | GPU: NVIDIA GeForce RTX 4060 Laptop GPU


In [2]:

PROJECT_ROOT = Path.cwd().parent.resolve()
DATA_YOLO    = (PROJECT_ROOT / "data" / "Data_YOLO").resolve()

IMG_TRAIN_DIR = DATA_YOLO / "images" / "train"
LAB_TRAIN_DIR = DATA_YOLO / "labels" / "train"
IMG_VAL_DIR   = DATA_YOLO / "images" / "val"
LAB_VAL_DIR   = DATA_YOLO / "labels" / "val"

print("PROJECT_ROOT:", PROJECT_ROOT)
print("DATA_YOLO   :", DATA_YOLO)
print("IMG_TRAIN   :", IMG_TRAIN_DIR, "exists:", IMG_TRAIN_DIR.exists())
print("LAB_TRAIN   :", LAB_TRAIN_DIR, "exists:", LAB_TRAIN_DIR.exists())
print("IMG_VAL     :", IMG_VAL_DIR,   "exists:", IMG_VAL_DIR.exists())
print("LAB_VAL     :", LAB_VAL_DIR,   "exists:", LAB_VAL_DIR.exists())

assert IMG_TRAIN_DIR.exists() and LAB_TRAIN_DIR.exists(), "Thiếu images/labels train – kiểm tra lại đường dẫn."


PROJECT_ROOT: /mnt/d/Defect-Detection
DATA_YOLO   : /mnt/d/Defect-Detection/data/Data_YOLO
IMG_TRAIN   : /mnt/d/Defect-Detection/data/Data_YOLO/images/train exists: True
LAB_TRAIN   : /mnt/d/Defect-Detection/data/Data_YOLO/labels/train exists: True
IMG_VAL     : /mnt/d/Defect-Detection/data/Data_YOLO/images/val exists: True
LAB_VAL     : /mnt/d/Defect-Detection/data/Data_YOLO/labels/val exists: True


In [3]:

VAL_RATIO = 0.15 

def list_images(folder: Path, exts=(".jpg",".jpeg",".png",".bmp",".tif",".tiff")):
    files = []
    for ext in exts:
        files += list(folder.glob(ext))
    return files

def ensure_val_split():
    if IMG_VAL_DIR.exists() and LAB_VAL_DIR.exists():
        print(" Val folders đã tồn tại. Bỏ qua tạo val.")
        return
    print(" Không tìm thấy val -> Tạo val với tỷ lệ", VAL_RATIO)
    IMG_VAL_DIR.mkdir(parents=True, exist_ok=True)
    LAB_VAL_DIR.mkdir(parents=True, exist_ok=True)

    train_imgs = list_images(IMG_TRAIN_DIR)
    if len(train_imgs) == 0:
        raise RuntimeError("Không có ảnh nào trong images/train.")

    set_seed(42)
    n_val = max(1, int(len(train_imgs) * VAL_RATIO))
    val_imgs = set(random.sample(train_imgs, n_val))

    moved = 0
    for img_path in tqdm(train_imgs, desc="Creating val split (copy)"):
        stem = img_path.stem
        lab_path = LAB_TRAIN_DIR / f"{stem}.txt"
        if img_path in val_imgs:

            shutil.copy2(img_path, IMG_VAL_DIR / img_path.name)

            if lab_path.exists():
                shutil.copy2(lab_path, LAB_VAL_DIR / lab_path.name)
            moved += 1
    print(f"Đã copy {moved} ảnh sang val. (Train còn nguyên, không xóa.)")

ensure_val_split()
print("IMG_VAL exists:", IMG_VAL_DIR.exists(), "| LAB_VAL exists:", LAB_VAL_DIR.exists())


 Val folders đã tồn tại. Bỏ qua tạo val.
IMG_VAL exists: True | LAB_VAL exists: True


In [4]:

DATA_YAML_PATH = (PROJECT_ROOT / "configs" / "data" / "m1_data.yaml")
DATA_YAML_PATH.parent.mkdir(parents=True, exist_ok=True)

data_yaml = {
    "train": str(IMG_TRAIN_DIR.resolve()),
    "val":   str(IMG_VAL_DIR.resolve()),
    "nc": 9,
    "names": ['SH','SP','SC','OP','MB','HB','CS','CFO','BMFO']
}
with open(DATA_YAML_PATH, "w") as f:
    yaml.dump(data_yaml, f)

print(" Wrote:", DATA_YAML_PATH)
print("train ->", data_yaml["train"])
print("val   ->", data_yaml["val"])


 Wrote: /mnt/d/Defect-Detection/configs/data/m1_data.yaml
train -> /mnt/d/Defect-Detection/data/Data_YOLO/images/train
val   -> /mnt/d/Defect-Detection/data/Data_YOLO/images/val


In [5]:

def stem_set(folder, exts=(".jpg",".jpeg",".png",".bmp",".tif",".tiff")):
    return {p.stem for ext in exts for p in folder.glob(ext)}

def txt_set(folder): 
    return {p.stem for p in folder.glob("*.txt")}

def report_split(img_dir, lab_dir, name="train"):
    imgs = stem_set(img_dir)
    labs = txt_set(lab_dir)
    miss_lab = sorted(imgs - labs)
    miss_img = sorted(labs - imgs)
    print(f"[{name}] images={len(imgs)}, labels={len(labs)}, missing_label={len(miss_lab)}, missing_image={len(miss_img)}")
    if miss_lab[:10]: print("  sample missing_label:", miss_lab[:10])
    if miss_img[:10]: print("  sample missing_image:", miss_img[:10])

report_split(IMG_TRAIN_DIR, LAB_TRAIN_DIR, "train")
report_split(IMG_VAL_DIR,   LAB_VAL_DIR,   "val")


[train] images=0, labels=8208, missing_label=0, missing_image=8208
  sample missing_image: ['0003730', '0003759', '0004469', '0004990', '0006194', '0012810', '0019339', '00320', '0032155', '0032227']
[val] images=0, labels=2051, missing_label=0, missing_image=2051
  sample missing_image: ['0014089', '0032075', '0032448', '0034013', '0034036', '0044633', '0045409', '0046436', '0048857', '0058315']


In [6]:

def peek_label_format_any(label_dir: Path, max_files=50):
    files = list(label_dir.glob("*.txt"))[:max_files]
    if not files: return None
    seg_votes, det_votes = 0, 0
    for f in files:
        lines = [ln for ln in f.read_text().strip().splitlines() if ln.strip()]
        if not lines: continue
        parts = lines[0].split()
        if len(parts) > 5: seg_votes += 1
        elif len(parts) == 5: det_votes += 1
    if seg_votes==0 and det_votes==0: return None
    return "SEG" if seg_votes >= det_votes else "DET"

fmt_train = peek_label_format_any(LAB_TRAIN_DIR)
fmt_val   = peek_label_format_any(LAB_VAL_DIR)
fmt = fmt_train or fmt_val
IS_SEG = (fmt == "SEG")
print("Label format:", fmt or "UNKNOWN", f"(train={fmt_train}, val={fmt_val})")


Label format: DET (train=DET, val=DET)


In [7]:

MODEL_SIZE = "s"      
EPOCHS     = 50
PATIENCE   = 20
IMG_SIZE   = 960
BATCH      = -1     
DEVICE     = "0"
WORKERS    = 4
PRETRAINED = True
COS_LR     = True
SEED       = 42


OUT_ROOT = PROJECT_ROOT / "artifacts" / "m1" / "runs"
TS       = datetime.now().strftime("%Y%m%d_%H%M%S")
RUN_NAME = f"m1_yolov8{MODEL_SIZE}_{'seg' if IS_SEG else 'det'}_{TS}"
OUT_DIR  = OUT_ROOT / RUN_NAME
OUT_DIR.mkdir(parents=True, exist_ok=True)


snap = {
    "ultralytics_version": ULTRA_VER, "torch": torch.__version__, "cuda": torch.version.cuda,
    "python": sys.version, "project_root": str(PROJECT_ROOT), "data_yaml": str(DATA_YAML_PATH),
    "run_name": RUN_NAME,
    "params": {"model_size": MODEL_SIZE,"epochs":EPOCHS,"patience":PATIENCE,"imgsz":IMG_SIZE,
               "batch":BATCH,"device":DEVICE,"workers":WORKERS,"pretrained":PRETRAINED,
               "cos_lr":COS_LR,"seed":SEED}
}
(OUT_DIR / "curves").mkdir(parents=True, exist_ok=True)
(OUT_DIR / "qualitative").mkdir(parents=True, exist_ok=True)
with open(OUT_DIR / "train_args.json","w") as f: json.dump(snap,f,indent=2)

try:
    req = subprocess.check_output([sys.executable,"-m","pip","freeze"], text=True)
    (OUT_DIR / "requirements.txt").write_text(req)
except Exception: pass

try:
    git_rev = subprocess.check_output(["git","rev-parse","HEAD"], text=True).strip()
    (OUT_DIR / "git_commit.txt").write_text(git_rev + "\n")
except Exception: pass

print("Artifacts will be saved to:", OUT_DIR)


Artifacts will be saved to: /mnt/d/Defect-Detection/artifacts/m1/runs/m1_yolov8s_det_20251104_074039


In [8]:

set_seed(SEED)
task = "segment" if IS_SEG else "detect"
ckpt = (f"yolov8{MODEL_SIZE}-seg.pt" if IS_SEG else f"yolov8{MODEL_SIZE}.pt") if PRETRAINED \
       else (f"yolov8{MODEL_SIZE}-seg.yaml" if IS_SEG else f"yolov8{MODEL_SIZE}.yaml")

print("Task:", task, "| Model:", ckpt)
model = YOLO(ckpt)

results = model.train(
    data=str(DATA_YAML_PATH),
    task=task,
    epochs=EPOCHS,
    patience=PATIENCE,
    imgsz=IMG_SIZE,
    batch=BATCH,
    device=DEVICE,
    workers=WORKERS,
    pretrained=PRETRAINED,
    cos_lr=COS_LR,
    seed=SEED,
    save=True,
    name=RUN_NAME,
    project=str(OUT_ROOT),
    verbose=True,
)


Task: detect | Model: yolov8s.pt


100%|██████████| 21.5M/21.5M [00:01<00:00, 13.8MB/s]


New https://pypi.org/project/ultralytics/8.3.224 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.0 🚀 Python-3.11.8 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/mnt/d/Defect-Detection/configs/data/m1_data.yaml, epochs=50, time=None, patience=20, batch=-1, imgsz=960, save=True, save_period=-1, cache=False, device=0, workers=4, project=/mnt/d/Defect-Detection/artifacts/m1/runs, name=m1_yolov8s_det_20251104_0740392, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=42, deterministic=True, single_cls=False, rect=False, cos_lr=True, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False,

100%|██████████| 6.25M/6.25M [00:00<00:00, 7.18MB/s]


AMP: checks passed ✅
AutoBatch: Computing optimal batch size for imgsz=960 at 60.0% CUDA memory utilization.
AutoBatch: CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU) 8.00G total, 0.13G reserved, 0.12G allocated, 7.75G free
      Params      GFLOPs  GPU_mem (GB)  forward (ms) backward (ms)                   input                  output
     9842443       53.08         0.667         99.89         223.6        (1, 3, 960, 960)                    list
     9842443       106.2         1.105         30.21         34.09        (2, 3, 960, 960)                    list
     9842443       212.3         2.135          46.6          43.7        (4, 3, 960, 960)                    list
     9842443       424.6         4.326         75.83         80.16        (8, 3, 960, 960)                    list
     9842443       849.3         8.305         163.2         292.1       (16, 3, 960, 960)                    list
AutoBatch: Using batch-size 8 for CUDA:0 4.48G/8.00G (56%) ✅


...

      50/50      4.41G      1.206     0.6931      1.203         19        960: 100%|██████████| 1026/1026 [03:44<00:00,  4.57it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 129/129 [00:27<00:00,  4.78it/s]

                   all       2051       4092      0.799      0.792      0.836      0.497



50 epochs completed in 3.724 hours.
Optimizer stripped from /mnt/d/Defect-Detection/artifacts/m1/runs/m1_yolov8s_det_20251104_0740392/weights/last.pt, 20.0MB
Optimizer stripped from /mnt/d/Defect-Detection/artifacts/m1/runs/m1_yolov8s_det_20251104_0740392/weights/best.pt, 20.0MB

Validating /mnt/d/Defect-Detection/artifacts/m1/runs/m1_yolov8s_det_20251104_0740392/weights/best.pt...
Ultralytics 8.3.0 🚀 Python-3.11.8 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
Model summary (fused): 186 layers, 9,831,147 parameters, 0 gradients, 23.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 129/129 [00:27<00:00,  4.74it/s]


                   all       2051       4092      0.816      0.775      0.838        0.5
                    SH        126        169      0.833      0.852      0.888      0.578
                    SP        430        929      0.868      0.731      0.844      0.395
                    SC        245        285      0.813       0.78      0.844      0.519
                    OP        274        338      0.802      0.837      0.891      0.543
                    MB        391        546      0.857      0.747      0.821      0.393
                    HB        271        608      0.933      0.936      0.984      0.842
                    CS        279        448      0.697      0.596      0.689      0.364
                   CFO        309        423      0.704      0.651      0.701      0.394
                  BMFO        305        346      0.841      0.842      0.882      0.469
Speed: 0.5ms preprocess, 6.9ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to /mnt/d/Defe

In [10]:

from pathlib import Path
import shutil, json


save_dir = None
try:
    save_dir = Path(model.trainer.save_dir).resolve()
    print("trainer.save_dir:", save_dir)
except Exception as e:
    print("No model.trainer.save_dir -> will scan OUT_ROOT")

if save_dir is None or not (save_dir / "weights").exists():
    candidates = [p for p in Path(OUT_ROOT).glob("*") if (p / "weights").exists()]
    if not candidates:
        raise FileNotFoundError(f"Không tìm thấy thư mục run có 'weights' trong {OUT_ROOT}")
    save_dir = max(candidates, key=lambda p: p.stat().st_mtime)
    print("Picked newest run:", save_dir)

weights_dir = save_dir / "weights"
best_pt = weights_dir / "best.pt"
last_pt = weights_dir / "last.pt"

OUT_DIR.mkdir(parents=True, exist_ok=True)
dest_best = OUT_DIR / "best.pt"
dest_last = OUT_DIR / "last.pt"

picked_src = None
if best_pt.exists():
    shutil.copy2(best_pt, dest_best)
    picked_src = dest_best
    if last_pt.exists():
        shutil.copy2(last_pt, dest_last)
    print("Copied best.pt (and last.pt if present).")
elif last_pt.exists():

    shutil.copy2(last_pt, dest_best)
    picked_src = dest_best
    print(" best.pt không có, dùng tạm last.pt -> best.pt")
else:
    files = [p.name for p in weights_dir.glob("*.pt")]
    raise FileNotFoundError(f"Không thấy best.pt/last.pt trong {weights_dir}. Có các file: {files}")

print("Picked weight:", picked_src)


picked_model = YOLO(str(picked_src))
metrics = picked_model.val(task=task, imgsz=IMG_SIZE, plots=True, device=DEVICE)
metrics_dict = {k: float(v) for k, v in metrics.results_dict.items()}
with open(OUT_DIR / "metrics.json", "w") as f:
    json.dump(metrics_dict, f, indent=2)

rescsv = save_dir / "results.csv"
if rescsv.exists():
    shutil.copy2(rescsv, OUT_DIR / "results.csv")

for fname in ["PR_curve.png", "confusion_matrix.png", "labels_correlogram.jpg"]:
    src = save_dir / fname
    if src.exists():
        (OUT_DIR / "curves").mkdir(parents=True, exist_ok=True)
        shutil.copy2(src, OUT_DIR / "curves" / fname)

print(" Saved weights/metrics/curves to:", OUT_DIR)


trainer.save_dir: /mnt/d/Defect-Detection/artifacts/m1/runs/m1_yolov8s_det_20251104_0740392
✅ Copied best.pt (and last.pt if present).
Picked weight: /mnt/d/Defect-Detection/artifacts/m1/runs/m1_yolov8s_det_20251104_074039/best.pt
Ultralytics 8.3.0 🚀 Python-3.11.8 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
Model summary (fused): 186 layers, 9,831,147 parameters, 0 gradients, 23.4 GFLOPs


val: Scanning /mnt/d/Defect-Detection/data/Data_YOLO/labels/val.cache... 2051 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2051/2051 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 129/129 [00:40<00:00,  3.19it/s]


                   all       2051       4092      0.817      0.775      0.838        0.5
                    SH        126        169      0.833      0.852      0.887      0.578
                    SP        430        929      0.868       0.73      0.843      0.394
                    SC        245        285      0.817      0.782      0.845      0.518
                    OP        274        338      0.802      0.837      0.891      0.544
                    MB        391        546      0.857      0.747      0.824      0.394
                    HB        271        608      0.933      0.936      0.984      0.842
                    CS        279        448      0.699      0.596      0.688      0.363
                   CFO        309        423      0.704      0.653      0.702      0.394
                  BMFO        305        346      0.841      0.842      0.882      0.469
Speed: 0.7ms preprocess, 12.6ms inference, 0.0ms loss, 1.3ms postprocess per image
Results saved to /mnt/d/Def

In [11]:

VAL_DIR = Path(data_yaml["val"])
val_imgs = []
for ext in ("*.jpg","*.jpeg","*.png","*.bmp","*.tif","*.tiff"):
    val_imgs += list(VAL_DIR.glob(ext))
val_imgs = sorted(val_imgs)[:16]

def overlay_prediction_bgr(im_bgr, res):
    out = im_bgr.copy()
    # boxes
    if getattr(res,"boxes",None) is not None and res.boxes is not None:
        for b in res.boxes.xyxy.cpu().numpy():
            x1,y1,x2,y2 = map(int,b); cv2.rectangle(out,(x1,y1),(x2,y2),(0,255,0),2)
    # masks 
    if getattr(res,"masks",None) is not None and res.masks is not None:
        masks = res.masks.data.cpu().numpy()
        for m in masks:
            m8 = (m*255).astype(np.uint8)
            cnts,_ = cv2.findContours(m8, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            cv2.drawContours(out, cnts, -1, (255,0,0), 2)
            overlay = out.copy()
            overlay[m8>0] = (0.5*overlay[m8>0] + 0.5*np.array([255,0,0])).astype(np.uint8)
            out = overlay
    return out

(OUT_DIR / "qualitative").mkdir(parents=True, exist_ok=True)
for p in tqdm(val_imgs, desc="Overlays"):
    im = cv2.imread(str(p))
    if im is None: continue
    res = model.predict(source=str(p), imgsz=IMG_SIZE, device=DEVICE, verbose=False)[0]
    over = overlay_prediction_bgr(im, res)
    cv2.imwrite(str(OUT_DIR / "qualitative" / p.name), over)

print("Saved overlays to:", OUT_DIR / "qualitative")


Overlays: 100%|██████████| 16/16 [00:02<00:00,  7.37it/s]

Saved overlays to: /mnt/d/Defect-Detection/artifacts/m1/runs/m1_yolov8s_det_20251104_074039/qualitative


In [12]:

export_paths = {}
onnx_path = model.export(format="onnx", imgsz=IMG_SIZE, simplify=True)
if onnx_path:
    onnx_path = Path(onnx_path)
    shutil.copy2(onnx_path, OUT_DIR / onnx_path.name)
    export_paths["onnx"] = onnx_path.name
    print("Exported ONNX:", OUT_DIR / onnx_path.name)

with open(OUT_DIR / "export_paths.json","w") as f:
    json.dump(export_paths, f, indent=2)


Ultralytics 8.3.0 🚀 Python-3.11.8 torch-2.5.1+cu121 CPU (AMD Ryzen 7 7735HS with Radeon Graphics)

PyTorch: starting from '/mnt/d/Defect-Detection/artifacts/m1/runs/m1_yolov8s_det_20251104_0740392/weights/best.pt' with input shape (1, 3, 960, 960) BCHW and output shape(s) (1, 13, 18900) (19.1 MB)
requirements: Ultralytics requirement ['onnxslim==0.1.34'] not found, attempting AutoUpdate...

requirements: AutoUpdate success ✅ 11.9s, installed 1 package: ['onnxslim==0.1.34']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect


ONNX: starting export with onnx 1.19.1 opset 19...
ONNX: slimming with onnxslim 0.1.34...
ONNX: export success ✅ 20.5s, saved as '/mnt/d/Defect-Detection/artifacts/m1/runs/m1_yolov8s_det_20251104_0740392/weights/best.onnx' (37.9 MB)

Export complete (23.8s)
Results saved to /mnt/d/Defect-Detection/artifacts/m1/runs/m1_yolov8s_det_20251104_0740392/weights
Predict:         yolo predict task=detect model=/mnt/d/Defect-Detection/artifacts/m1/r

In [13]:

print("=== DONE ===")
for p in ["best.pt","metrics.json","results.csv","export_paths.json"]:
    fp = OUT_DIR / p
    print(f"{p:18s} ->", "OK" if fp.exists() else "MISSING", "|", fp)
print("Curves     ->", OUT_DIR / "curves")
print("Qualitative->", OUT_DIR / "qualitative")
print("Run folder ->", OUT_DIR)
print("Task       ->", "SEG (mask)" if IS_SEG else "DET (bbox) — nếu cần đo lường, hãy sinh mask rồi train lại SEG")


=== DONE ===
best.pt            -> OK | /mnt/d/Defect-Detection/artifacts/m1/runs/m1_yolov8s_det_20251104_074039/best.pt
metrics.json       -> OK | /mnt/d/Defect-Detection/artifacts/m1/runs/m1_yolov8s_det_20251104_074039/metrics.json
results.csv        -> OK | /mnt/d/Defect-Detection/artifacts/m1/runs/m1_yolov8s_det_20251104_074039/results.csv
export_paths.json  -> OK | /mnt/d/Defect-Detection/artifacts/m1/runs/m1_yolov8s_det_20251104_074039/export_paths.json
Curves     -> /mnt/d/Defect-Detection/artifacts/m1/runs/m1_yolov8s_det_20251104_074039/curves
Qualitative-> /mnt/d/Defect-Detection/artifacts/m1/runs/m1_yolov8s_det_20251104_074039/qualitative
Run folder -> /mnt/d/Defect-Detection/artifacts/m1/runs/m1_yolov8s_det_20251104_074039
Task       -> DET (bbox) — nếu cần đo lường, hãy sinh mask rồi train lại SEG
